In [1]:
# Upgrade pip
!pip install --upgrade pip

# Install PyTorch (Ensure compatibility with your hardware, e.g., CUDA or MPS)
!pip install torch torchvision torchaudio

# Install additional dependencies
!pip install fvcore  # For FLOPs calculation

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61406 sha256=aac3bb0e3391dab2a410860631a93ca807e73d2c40dea9590bbaeafc25201dd3
  Stored in directory: /root/.cache/pip/wheels/65/71/95/3b8fde5c65c6e4a806e0867c1651dcc71a1cb2f3430e8f355f
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31532 sha256=4d94198c7e3aa3a47f2d3b37178e20c1bbd76cf284f49968311cdf42c513fd32
  Stored in directory: /root/.cache/pip/wheels/ba/5e/16/6117f8fe7e9c0c161a795e10d94645ebcf301ccbd01f66d8ec
Successfully built fvcore iopath


In [2]:
# Import necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models
import time
import os
import matplotlib.pyplot as plt

In [3]:
import utils.dependencies as utils
import utils.metrics as metrics

In [4]:
# Define device
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using CUDA GPU")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
    print("Using Apple Silicon GPU (MPS)")
else:
    device = torch.device("cpu")
    print("Using CPU")

Using CUDA GPU


In [5]:
# Define data loaders
train_loader, val_loader, test_loader = utils.get_data_loaders(dataset_name='CIFAR10', data_dir='./data',
                                                               batch_size=128, num_workers=8, pin_memory=True,
                                                               valid_split=0.1, seed=42)

# Initialize the model
model = models.efficientnet_b0() 

# Modify the classifier to match CIFAR-10 (10 classes)
num_ftrs = model.classifier[1].in_features
model.classifier[1] = nn.Linear(num_ftrs, 10)

# Move the model to the specified device
model = model.to(device)

# Print the model architecture
print(model)

Files already downloaded and verified
Files already downloaded and verified
EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
          

In [6]:
# Freeze all layers except the classifier
for name, param in model.named_parameters():
    if "classifier" not in name:
        param.requires_grad = False
    
# Define the layers to unfreeze (last two blocks)
layers_to_unfreeze = ['features.5', 'features.6', 'features.7']

# Unfreeze the specified layers
utils.unfreeze_layers(model, layers_to_unfreeze)

# Define optimizer to include only trainable parameters
optimizer = optim.SGD(
    filter(lambda p: p.requires_grad, model.parameters()),
    lr=0.01,  
    momentum=0.9,
    weight_decay=5e-4
)

# Define a learning rate scheduler for fine-tuning
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

In [7]:
# Training loop
num_epochs = 10
best_val_acc = 0.0

for epoch in range(1, num_epochs + 1):
    print(f"--- Epoch {epoch} ---")
    
    # Train
    train_loss, train_acc = utils.train_epoch(model, device, train_loader, optimizer)
    
    # Validate
    val_loss, val_acc = utils.validate_epoch(model, device, val_loader)
    
    # Step the scheduler
    scheduler.step()
    
    # Save the best model
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), 'best_model.pth')
        print(f"Best model saved with Val Acc: {best_val_acc:.2f}%\n")
    else:
        print("No improvement this epoch.\n")

--- Epoch 1 ---


Exception in thread Thread-5 (_pin_memory_loop):
Traceback (most recent call last):
  File "/usr/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py", line 761, in run_closure
    _threading_Thread_run(self)
  File "/usr/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/_utils/pin_memory.py", line 54, in _pin_memory_loop
    do_one_step()
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/_utils/pin_memory.py", line 31, in do_one_step
    r = in_queue.get(timeout=MP_STATUS_CHECK_INTERVAL)
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/multiprocessing/queues.py", line 122, in get
    return _ForkingPickler.loads(res)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/multiprocessing/reductions.py", 

KeyboardInterrupt: 

In [ ]:
# Load the best model
model.load_state_dict(torch.load('best_model.pth', map_location=device))
print("Loaded the best model based on validation accuracy.")

# Test the model
test_loss, test_acc = utils.test_model(model, device, test_loader, criterion)
print(f"Final Test Loss: {test_loss:.4f} | Final Test Accuracy: {test_acc:.2f}%")

# Generate and save metrics plots
metrics.generate_and_save_metrics(model, device, input_size=(1,3,224,224), 
                                  save_dir='metrics_plots', model_name='efficientnet_b0_base')

In [ ]:
!pip install thop


In [ ]:
import utils.metrics as metrics
# Define loss function
criterion = nn.CrossEntropyLoss()
# Generate and save metrics for pruned model after fine-tuning
metrics.generate_and_save_metrics(
    model, 
    device, 
    test_loader, 
    criterion, 
    model_name='efficientnet_b0_pruned_post_finetune', 
    pruning_ratio=0.8, 
    description='Pruned model after fine-tuning'
)